# 1. File description 

To feed a model for it to predict a bot from a legit users I need the data to be labelled meaning each user in the dataset needs to have a 1 or 0 for bot/not bot. This file is meant to do so.

To label the data, multiple techniques are gonna be used and are divided in three. Manual, semi-manual and automatic labelling. 

Automatic: 
* Look at bot labelled and check for other users with the same photo. From previous iteration of the project, I have a folder with about 800 of bots photos that I previously found manually. Now, I just check if a user's profile pic is similar to any, if yes, it's a bot. That's the easiest way and the only one where I don't have to check other details of the bot as if two users have the same photo and one is a bot, the other is too.

Semi-manual:
* Look at the bots having a scam domain.
* Use NSFW pre-trained models to get images score for Neutral, Drawing, Hentai, Sexual, Porn and looking at the profile that have a high score for porn + sexy as most bots have nude photos
* Look at a subset of data where I belive bots are
    * Around 200 likes on their comment
    * 12 or less photos
    * Always have a link in their bio
* Find same/similar comments from labelled users 
* Pattern in Bio/description
* Look at the keyword of the flagged bots in comments and profile, look at profile that are similar to bots. 
* After labelling abunch of users, build a weak model to help in the labelling by making predictions. As long as the model is better than random, it'll be helpful.



## 0. Import module and setting notebook preferences

### 0.1 Import modules

In [1]:
import src.useful as su

import pandas as pd 
import numpy as np
import sqlite3 

import plotly.io as pio

from imagededup.methods import PHash
from PIL import Image, ImageDraw, ImageFont, UnidentifiedImageError

import os
import shutil
import joblib

from tqdm import tqdm 

from IPython.display import Image as IImage, display, clear_output

ModuleNotFoundError: No module named 'demoji'

### 0.2 Notebook preferences

In [ ]:
pd.set_option("display.max_columns", None)
pio.templates.default = "plotly_dark"

# Creating SQL database to store all the data for the project
database = "data/main_database.sqlite"
con = sqlite3.connect(database)

## 1. Manual labelling 

To label the data manually, the process has to be fast, I need to see all the information about the users condensed in one summary. For that, I chose to create for each user an image with their attributes and their photos. I'll then create a script to open one image by one, and then chose bot/not bot/maybe bot with keyboard arrows. 



In [ ]:
# Load the data and display only the one from a 
query = '''
select 
    * 
from clean_comments_users_last12
'''

df_main = pd.read_sql_query(query, con)

In [ ]:
# Code to create an empty dataframe for the table labels

# df = pd.read_csv('/Users/marclamy/Desktop/code/all ig bot folders/igbot_final/data/labelled_usernames_method2_temp.csv')
# df.head(0).to_sql('labels', con, if_exists='append', index=False)

### 1.1 Generate summary

I'm going to use the profile of a bot for the image demo. The account is still live but hasn't been active for about 9 months, at last the last comment on their post is. This is the developed function from [generate_summary.py](https://github.com/marclelamy/instagram_bot_classification/blob/main/3.1_generate_summary.py)

First, create a blank image, define font.

In [ ]:
global_image = Image.new("RGB", (1300, 450), "black")
draw = ImageDraw.Draw(global_image)
font = ImageFont.truetype("assets/OpenSans-Light.ttf", 15)

global_image

Add the profile picture 

In [ ]:
username = 'accurate-eccentric-lion'
df_user = df_main[df_main["username"]==username].reset_index(drop=True)

# Open user pp and add it to the global image
try:
    path = f"data/photos/user_profile_pictures/{username}_pp_user_photo.png"
    profile_pic = Image.open(path)
    global_image.paste(profile_pic, (250, 0))
except (FileNotFoundError, UnidentifiedImageError): 
    pass

global_image

Adding twelve other photos 

In [ ]:
# Creating a list of position to add the photos on the global image
positions = []
for x in range(850, 1300, 150):
    for y in range(0, 450, 150):
        positions.append((x, y))


# Loop through each image and add it to the global image
for image_num in range(12):
    path = f"data/photos/user_last_12_posts/{username}_{str(image_num)}_user_photo.png"

    try:
        if os.path.exists(path):
            img = Image.open(path)
            if image_num < 9:
                global_image.paste(img.resize((150, 150)), positions[image_num])
            elif image_num == 9:
                global_image.paste(img.resize((150, 150)), (400, 0))
            elif image_num == 10:   
                global_image.paste(img.resize((150, 150)), (550, 0))
            elif image_num == 11:
                global_image.paste(img.resize((150, 150)), (700, 0))
    except (OSError, UnidentifiedImageError) as e: # For truncated image file
        pass

global_image

Adding screenshot of the landing page of the url in bio

In [ ]:
# Open screenshot, resize it, add it to global_image 
path = f"data/photos/bio_url_screenshot/{username}_website_photo.png"
try:
    screenshot = Image.open(path)
    global_image.paste(screenshot.resize((500, 300)), (350, 150))
except (UnidentifiedImageError, FileNotFoundError):
    pass

global_image

In [ ]:
df_user

In [ ]:
# Add basic information 
username = df_user.loc[0, "username"]
draw.text((10, 5), f"Username: {username}", font=font, fill="white")

follower_count = df_user.loc[0, "follower_count"]
color = "white" if follower_count > 800 else "red"
draw.text((10, 25), f"Followers: {follower_count:,}", font=font, fill=color)

follow_count = df_user.loc[0, "follow_count"]
draw.text((10, 45), f"Following: {follow_count:,}", font=font, fill="white")

post_count = df_user.loc[0, "post_count"]
color = "white" if post_count > 16 else "red"
draw.text((10, 65), f"Post count: {post_count}", font=font, fill=color)

video_count = df_user.loc[0, "video_count"]
color = "white" if video_count > 0 else "red"
draw.text((10, 85), f"Video count: {video_count}", font=font, fill=color)

comment_likes = round(df_user["comment_likes"].mean())
color = "white" if 200 <= comment_likes < 450 else "red"
draw.text((10, 105), f"Comment_likes: {comment_likes}", font=font, fill="white")

domain = df_user.loc[0, "domain"]
draw.text((10, 125), f"{domain}", font=font, fill="white")

biography = df_user.loc[0, "biography"]
draw.text((10, 145), f"{biography}", font=font, fill="white")


global_image

Save image

In [ ]:
# summary_path = f"data/photos/image_summary/{username}_image_summary.png"
# global_image.save(summary_path)

### 1.2 Manual labelling one by one

This is the process of labelling the users one by one, twice to eliminate human error. For this I made a script to open each image summary generated in the precedent step and label it with the arrows: `{'right': 'bot', 'up': 'legit', 'down': 'maybe'}`.

Look up for [3.2_manual_labelling.py](https://github.com/marclelamy/instagram_bot_classification/blob/main/3.2_manual_labelling.py)

## 2. Semi-manual labelling

### Image dupes

In [ ]:
# # list all the photos folders
# last12_folder = 'data/photos/user_last_12_posts'
# pp_folder = 'data/photos/user_profile_pictures'
# allphotos_folder = 'data/photos/all_photos'


# phasher = PHash()
# folder_paths = [pp_folder, last12_folder, allphotos_folder]
# # folder_paths = ['test_folder_photos']
# for folder_path in tqdm(folder_paths): 
#     file_name = folder_path.split('/')[-1]
#     print(f'data/photos/{file_name}_encodings.pkl')
#     if os.path.isfile(f'data/photos/{file_name}_encodings.pkl'):
#         encodings = joblib.load(f'data/photos/{file_name}_encodings.pkl')
#     else: 
#         encodings = phasher.encode_images(image_dir=folder_path)
    
#     if os.path.isfile(f'data/photos/{file_name}_duplicates.pkl') == False:
#         duplicates = phasher.find_duplicates(encoding_map=encodings)
#         joblib.dump(duplicates, f'data/photos/{file_name}_duplicates.pkl')


In [ ]:
# pp_path_duplicated = joblib.load(f'data/photos/user_profile_pictures_light_duplicates.pkl')
# last12_path_duplicated = joblib.load('data/photos/user_profile_pictures_light_duplicates.pkl')
all_path_duplicated = joblib.load('data/photos/all_photos_duplicates.pkl')

# print(len(all_path_duplicated), len([key for key, value in all_path_duplicated.items() if len(value) > 0]), len([[key, value] for key, values in all_path_duplicated.items() for value in values]))


# def dupeless (dupes): 
#     for key, values in list(dupes.items()):
#         if key in dupes.keys():
#             if len(values) == 0:
#                 del dupes[key]
#                 continue
#             for value in values:
#                 if value in dupes.keys():
#                     del dupes[value]

#     return dupes 

# du = dupeless(all_path_duplicated)
# import sys
# print(sys.getsizeof(du), sys.getsizeof(du) / sys.getsizeof(all_path_duplicated))

In [ ]:
# len([[key, value] for key, values in all_path_duplicated.items() for value in values]), len([[key, value] for key, values in du.items() for value in values])

In [ ]:
def dupeless (dupes): 
    for key, values in list(dupes.items()):
        if key in dupes.keys():
            if len(values) == 0:
                del dupes[key]
                continue
            for value in values:
                if value in dupes.keys():
                    del dupes[value]

    return dupes 

print(len(all_path_duplicated.keys()))
all_path_duplicated = dupeless(all_path_duplicated)
print(len(all_path_duplicated.keys()))


all_values = [''.join(sorted([key, value])) for key, values in all_path_duplicated.items() for value in values]
len(set(all_values)) == len(all_values)

In [ ]:
photos_dupes = [sorted([key, value]) for key, values in tqdm(all_path_duplicated.items()) for value in values]

Remove the type of the photo to only keep the username and remove the duplicates on the username/username_dupe subset

In [ ]:
# dict(pd.read_sql_query('select cooler_name, username from username_mapping', con).values)

In [ ]:
# username_mapping = dict(pd.read_sql_query('select username, cooler_name from username_mapping', con).values)

# def rmv_photo_type(text): 
#     original_text = text
#     for replacement in [f'_{num}_user_photo.png' for num in range(12)] + ['_pp_user_photo.png']:
#         text = text.replace(replacement, '')

#     return original_text.replace(text, username_mapping[text])


# for index, value in enumerate(photos_dupes):
#     photos_dupes[index] = [rmv_photo_type(value[0]), rmv_photo_type(value[1])]

In [ ]:
# Create photos dupes dataframe
df_photos_dupes = pd.DataFrame(photos_dupes, columns=['photo_name', 'photo_name_dupe'])

# Remove duplicate combinations
print(f'Before removing dupes: {df_photos_dupes.shape}')
df_photos_dupes = df_photos_dupes.drop_duplicates()
print(f'After removing dupes: {df_photos_dupes.shape}')

df_photos_dupes

#### Adding and ID for each group 

In [ ]:
# Adding an ID for all similar photos
photos_names = df_photos_dupes.groupby('photo_name', as_index=False).count().sort_values(['photo_name_dupe', 'photo_name'], ascending=False)['photo_name']
image_group_mapping = {photo_name: index for index, photo_name in enumerate(photos_names)}
df_photos_dupes['groupid'] = df_photos_dupes['photo_name'].apply(lambda x: image_group_mapping[x])
df_photos_dupes.to_sql('photos_dupes', con, if_exists='replace', index=False)

In [ ]:
df_photos_dupes.groupby('photo_name', as_index=False).count().sort_values(['photo_name_dupe', 'photo_name'], ascending=False)

In [ ]:
df_photos_dupes.query('photo_name == "abiding-beige-hound_pp_user_photo.png"')

#### X.x Creating function to see group photo and stor them properly

In [ ]:
# Show image of specific category
def show_photo_group(groupid):
    df_photos_dupes_group_category = df_photos_dupes.query(f'groupid == {groupid}')[['photo_name', 'photo_name_dupe']]
    photos_dupes_group_category = [x for l in df_photos_dupes_group_category.values for x in l]
    print(groupid, len(photos_dupes_group_category))
    return IImage(filename=f'data/photos/all_photos/' + photos_dupes_group_category[0])

# show_photo_group(36)

In [ ]:
def store_groupid_photo_together(category):
    df_photos_dupes_group_category = df_photos_dupes.query(f'groupid == {category}')[['photo_name', 'photo_name_dupe']]
    photos_dupes_group_category = {x for l in df_photos_dupes_group_category.values for x in l}
    # print(category, len(photos_dupes_group_category))

    # Check if folder exists, otherwise create it
    dir_copy = f'data/photos/categories/{category}/'
    if os.path.isdir(dir_copy):
        for file in su.list_files(dir_copy, path=True):
            os.remove(file)
    else:
        os.makedirs(dir_copy)

    # Copying each image into the new directory
    for photo in photos_dupes_group_category:
        try:
            shutil.copy('data/photos/all_photos/' + photo, dir_copy + photo)
        except Exception as e:
            pass
    
    shutil.copy('data/photos/all_photos/' + photo, 'data/photos/categories/00unique/' + str(category) + '.png')
    # display(IImage(filename=dir_copy + photo))


for groupid in range(5):
    # display(show_photo_group(groupid))
    store_groupid_photo_together(groupid)

In [ ]:
asdf

In [ ]:
category = 4554
df_photos__dupes_group_4554 = df_photos_dupes.query(f'photo_combo_group == {category}')[['photo_name', 'photo_name_dupe']]
photos__dupes_group_4554 = {x for l in df_photos__dupes_group_4554.values for x in l}

for photo in photos__dupes_group_4554:
    shutil.copy('data/photos/all_photos/' + photo, f'data/photos/categories/{category}/' + photo)

In [ ]:
photos__dupes_group_4554

In [ ]:
df_photos_dupes[['photo_name','photo_name_dupe']]

In [ ]:
df_photos_dupes = df_photos_dupes[['username', 'username_dupe', 'photo_name', 'photo_name_dupe']]
df_photos_dupes.to_sql('photos_dupe', con, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from clean_users', con)

### 0.0 Create file with photos

In [ ]:

np.random.shuffle(last12_photos_paths)
for path in tqdm(last12_photos_paths[:50000]): 
    shutil.copy(path, path.replace('data/photos/user_last_12_posts/', 'test_folder_photos/'))

In [ ]:
path.replace('data/photos/user_last_12_posts/', 'test_folder_photos/')

In [ ]:
len(list_files('test_folder_photos'))

## 3. Automatic labelling